In [1]:
!pip install python-dotenv
!pip install langchain
!pip install pypdf  # Biblioteca necesaria para cargar archivos PDF
!pip install -U langchain-community
!pip install -U langchain-openai
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.7 MB/s eta 0:00:00
  Attempting unin

In [2]:
with open('.env', 'w') as f:
    f.write("OPENAI_API_KEY=")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [5]:
# Ruta al directorio que contiene los archivos PDF
pdf_directory = "/content/drive/MyDrive/1_Proyectos/02_TFM/BaseConocimiento"
persist_directory = pdf_directory

In [6]:
from langchain.document_loaders import PyPDFLoader
import glob


# Cargar todos los archivos PDF en el directorio
loaders = [PyPDFLoader(filepath) for filepath in glob.glob(f"{pdf_directory}/*.pdf")]

# Cargar los documentos desde todos los PDFs
docs = []
for loader in loaders:
    docs.extend(loader.load())

# Verificación opcional: imprimir el número de documentos cargados
print(f"Total de documentos cargados: {len(docs)}")

Total de documentos cargados: 97


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configuración del splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)

# Dividir los documentos
splits = text_splitter.split_documents(docs)

# Verificar el número de fragmentos generados
print(f"Número total de fragmentos: {len(splits)}")

Número total de fragmentos: 152


In [8]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [9]:
from langchain.vectorstores import Chroma
!rm -rf ./docs/chroma  # remove old database files if any
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

152
